# ePSproc: auto-gen template development, Jupyter-runner version TIDY

12/01/20
- Updated headers for (nb)Sphinx HTML compatibility (all down one level).

10/01/20
- Updating with Euler labels.
- Incorporated changes from 16/12/19 version (unfinished).

05/12/19
- Updated with lmPlot() routine.

05/11/19
- Set for single template per ePS datafile.
- Outputs BLM-L and BLM-V, named as per input file + datestamp.

## Set-up

### Load modules

In [ ]:
import sys
import os
import numpy as np
import epsproc as ep

from datetime import datetime as dt
timeString = dt.now()

### Load data

In [ ]:
# File path only, from env var DATAFILE
# dataPath = os.getcwd()
dataFile = os.environ.get('DATAFILE', '')

In [ ]:
jobInfo = ep.headerFileParse(dataFile)
molInfo = ep.molInfoParse(dataFile)

In [ ]:
# Scan file(s) for various data types...

# For dir scan
# dataXS = ep.readMatEle(fileBase = dataPath, recordType = 'CrossSection')
# dataMatE = ep.readMatEle(fileBase = dataPath, recordType = 'DumpIdy')

# For single file
dataXS = ep.readMatEle(fileIn = dataFile, recordType = 'CrossSection')
dataMatE = ep.readMatEle(fileIn = dataFile, recordType = 'DumpIdy')


## Job & molecule info

In [ ]:
ep.jobSummary(jobInfo, molInfo);

## 1-photon ePS Cross-Sections
Plot 1-photon cross-sections and $beta_2$ parameters (for an unaligned ensemble) from ePS calculations. These are taken directly from the ePS output file, `CrossSection` segments. See the [ePS manual, `GetCro` command, for further details](https://www.chem.tamu.edu/rgroup/lucchese/ePolyScat.E3.manual/GetCro.html).

### Cross-sections by symmetry & type

Types correspond to:

- 'L': length gauge results.
- 'V': velocity gauge results.
- 'M': mixed gauge results.

Symmetries correspond to allowed ionizing transitions for the molecular point group (IRs typically corresponding to (x,y,z) polarization geometries), see the [ePS manual for a list of symmetries](https://www.chem.tamu.edu/rgroup/lucchese/ePolyScat.E3.manual/SymmetryLabels.html). Symmetry `All` corresponds to the sum over all allowed sets of symmetries.

Cross-section units are MBarn.

In [ ]:
# Plot cross sections using Xarray functionality
# Set here to plot per file - should add some logic to combine files.
for data in dataXS:
    daPlot = data.sel(XC='SIGMA')
    daPlot.plot.line(x='Eke', col='Type')

### $\beta_{2}$ by symmetry & type

Types & symmetries as per cross-sections.  Normalized $\beta_{2}$ paramters, dimensionless.

In [ ]:
# Repeat for betas
for data in dataXS:
    daPlot = data.sel(XC='BETA')
    daPlot.plot.line(x='Eke', col='Type')

## Dipole matrix elements
For 1-photon ionization. These are taken directly from ePS `DumpIdy` segments. See the [ePS manual, `DumpIdy` command, for further details](https://www.chem.tamu.edu/rgroup/lucchese/ePolyScat.E3.manual/DumpIdy.html).

In [ ]:
# Set threshold for significance, only matrix elements with abs values > thres % will be plotted
thres = 0.1

In [ ]:
# Plot for each fie
for data in dataMatE:
    # Plot with sensible defaults - all dims with lmPlot()
    
    # Plot only values > theshold
    daPlot, daPlotpd, legendList, gFig = ep.lmPlot(data, thres = thres, thresType = 'pc', figsize = (15,10))

    # Plot phases, with unwrap
    daPlot, daPlotpd, legendList, gFig = ep.lmPlot(data, thres = thres, thresType = 'pc', figsize = (15,10), pType='phaseUW')

## MFPADs

Calculated MF $\beta$ parameters, using ePS dipole matrix elements. These are calculated by `ep.mfblm()`, as a function of energy and polarization geometry. See [the ePSproc docs on `ep.mfblm()`](https://epsproc.readthedocs.io/en/latest/modules/epsproc.MFBLM.html) for further details, and [this demo notebook](https://epsproc.readthedocs.io/en/latest/ePSproc_BLM_calc_demo_Sept2019_rst/ePSproc_BLM_calc_demo_Sept2019.html).

In [ ]:
# Set pol geoms - these correspond to (z,x,y) in molecular frame (relative to principle/symmetry axis)
eAngs = ep.setPolGeoms()

In [ ]:
# Calculate for each fie & pol geom
# TODO - file logic, and parallelize
BLM = []
for data in dataMatE:
    BLM.append(ep.mfblmEuler(data, selDims = {'Type':'L'}, eAngs = eAngs, thres = thres, 
                             SFflag = True, verbose = 0))  # Run for all Eke, selected gauge only

In [ ]:
# Save BLM data - defaults to working dir and 'ep_timestamp' file
# TODO - testing for array/multiple file case
for data in BLM:
    fileName = dataFile + '_BLM-L_' + timeString.strftime('%Y-%m-%d_%H-%M-%S')
    ep.writeXarray(data, fileName = fileName)

In [ ]:
# Normalize and plot results
for BLMplot in BLM:
    # Plot unnormalized B00 only, real part
    # This is/should be in units of MBarn (TBC).
#     BLMplot.where(np.abs(BLMplot) > thres, drop = True).real.squeeze().sel({'l':0, 'm':0}).plot.line(x='Eke', col='Euler');
    BLMplot.XS.real.squeeze().plot.line(x='Eke', col='Euler');

    # Plot values normalised by B00 - now set in calculation function
    # Plot results with lmPlot(), ordering by Euler sets
    # Version with (semi-manual) Euler grouping
    daPlot, daPlotpd, legendList, gFig = ep.lmPlot(BLMplot.swap_dims({'Euler':'Labels'}), SFflag = False, eulerGroup = True,
                                                   thresType = 'pc', thres = thres,
                                                   plotDims = ('Labels','l','m'), 
                                                   figsize = (15,10))

## Error & consistency checks

In [ ]:
# Check SF values
for data in dataMatE:
    # Plot values, single plot
    data.SF.pipe(np.abs).plot.line(x='Eke')
    data.SF.real.plot.line(x='Eke')
    data.SF.imag.plot.line(x='Eke')
    
    # Plot values, facet plot
#     data.SF.pipe(np.abs).plot.line(x='Eke', col='Sym')


In [ ]:
# Compare calculated BLMs for L and V types (dafault above for L)

# Calculate for each fie & pol geom, and compare.
BLMv = []
BLMdiff = []
for n, data in enumerate(dataMatE):
    BLMv.append(ep.mfblmEuler(data, selDims = {'Type':'V'}, eAngs = eAngs, thres = thres, 
                             SFflag = True, verbose = 0))  # Run for all Eke, selected gauge only
    
    BLMdiff.append(BLM[n] - BLMv[n])
    BLMdiff[n]['dXS'] = BLM[n].XS - BLMv[n].XS  # Set XS too, dropped in calc above

    BLMdiff[n].attrs['dataType'] = 'matE'

In [ ]:
# Save BLM data - defaults to working dir and 'ep_timestamp' file
# TODO - testing for array/multiple file case
for data in BLMv:
    fileName = dataFile + '_BLM-V_' + timeString.strftime('%Y-%m-%d_%H-%M-%S')
    ep.writeXarray(data, fileName = fileName)

In [ ]:
# Normalize and plot results
for BLMplot in BLMv:
    # Plot unnormalized B00 only, real part
    # This is/should be in units of MBarn (TBC).
#     BLMplot.where(np.abs(BLMplot) > thres, drop = True).real.squeeze().sel({'l':0, 'm':0}).plot.line(x='Eke', col='Euler');
    BLMplot.XS.real.squeeze().plot.line(x='Eke', col='Euler');

    # Plot values normalised by B00 - now set in calculation function
    # Plot results with lmPlot(), ordering by Euler sets
    # Version with (semi-manual) Euler grouping
    daPlot, daPlotpd, legendList, gFig = ep.lmPlot(BLMplot.swap_dims({'Euler':'Labels'}), SFflag = False, eulerGroup = True,
                                                   thresType = 'pc', thres = thres,
                                                   plotDims = ('Labels','l','m'), 
                                                   figsize = (15,10))

In [ ]:
# Difference between 'L' and 'V' results
# NOTE - this currently drops XS

print('Differences, L vs. V gauge BLMs')

for BLMplot in BLMdiff:
    maxDiff = BLMplot.max()
    print(f'Max difference in BLMs (L-V): {0}', maxDiff.data)
    
    if np.abs(maxDiff) > thres:
        # Plot B00 only, real part
#         BLMplot.where(np.abs(BLMplot) > thres, drop = True).real.squeeze().sel({'l':0, 'm':0}).plot.line(x='Eke', col='Euler');
        BLMplot.dXS.real.squeeze().plot.line(x='Eke', col='Euler');

        # Plot values normalised by B00 - now set in calculation function
        # Plot results with lmPlot(), ordering by Euler sets
        # Version with (semi-manual) Euler grouping
        daPlot, daPlotpd, legendList, gFig = ep.lmPlot(BLMplot.swap_dims({'Euler':'Labels'}), SFflag = False, eulerGroup = True,
                                                       thresType = 'pc', thres = thres,
                                                       plotDims = ('Labels','l','m'), 
                                                       figsize = (15,10))


In [ ]:
# Check imaginary components - should be around machine tolerance.
print('Machine tolerance: ', np.finfo(float).eps)
for BLMplot in BLM:
    maxImag = BLMplot.imag.max()
    print(f'Max imaginary value: {0}', maxImag.data)
    
#     BLMplot.where(np.abs(BLMplot) > thres, drop = True).imag.squeeze().plot.line(x='Eke', col='Euler');
    
    BLMplot = ep.matEleSelector(BLMplot, thres=thres, dims = 'Eke')
    BLMplot.imag.squeeze().plot.line(x='Eke', col='Euler');

## Version info

### Original job details

In [ ]:
print(jobInfo['ePolyScat'][0])
print('Run: ' + jobInfo['Starting'][0].split('at')[1])

### ePSproc details

In [ ]:
templateVersion = '0.0.5'
templateDate = '10/01/20'

In [ ]:
%load_ext version_information

In [ ]:
%version_information epsproc, xarray

In [ ]:
print('Run: {}'.format(timeString.strftime('%Y-%m-%d_%H-%M-%S')))
host = !hostname
print('Host: {}'.format(host[0]))